In [ ]:
#SET PARAMS HERE
l = 5
species = 'ecoli'
real_com_file_all =  'ecocyc-complexes-w-dummy-names.txt' #'ecoli-protein-complexes-ext-genenames-w-names.tsv'

In [ ]:
#returns list of complexes in file with at least min_nodes proteins
def read_real_coms(file, min_nodes):
    coms = []
    with open(file) as file:
        for line in file:
            #print(line)
            comname,members = line.rstrip().split("\t")
            words = members.split()
            if len(words) >= min_nodes:
                coms.append([comname, words])
    return coms

In [ ]:
#returns list of communities/pseudo-cliques in file
def read_coms(file):
    coms = []
    with open(file) as file:
        for line in file:
            words = line.rstrip().split()
            coms.append(words)
    return coms

In [ ]:
def matching_score(set1, set2):
    """Calculates the matching score between two sets (e.g., a cluster and a complex)
    using the approach of Bader et al, 2001"""
    return len(set1.intersection(set2))**2 / (float(len(set1)) * len(set2))

def fraction_matched(reference, predicted, score_threshold=0.25):
    result = 0

    for id1, c1 in enumerate(reference):
        for id2, c2 in enumerate(predicted):
            score = matching_score(c1, c2)
            if score > score_threshold:
                result += 1
                break

    return result / len(reference)

In [ ]:
def compute_overlap(com1, com2):
    sum1 = 0
    sum2 = 0
    ratios = []
    for a in com1:
        maxCommon = []
        for b in com2:
            common = list(set(a) & set(b))
            if len(common) > len(maxCommon):
                maxCommon = common
        sum1 += len(maxCommon) #/len(a)
        sum2 += len(a)
    ratio = sum1/sum2 #sum1/len(com1)
    return ratio

In [ ]:
def f1(prec,rec):
    if prec == 0 and rec ==  0:
        return 0
    else:
        return 2*prec*rec/(prec+rec)

In [ ]:
#special cases are based on https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
def prec_rec_f1(real_coms, pred_coms):
    if len(real_coms) == 0 and len(pred_coms) == 0:
        return [1,1,1]
    elif len(real_coms) == 0 or len(pred_coms) == 0:
        return [0, 0, 0]
    else:  #when neither are zeros then normal process needs to be followed
        avg_recall = compute_overlap(real_coms, pred_coms)
        avg_precision = compute_overlap(pred_coms, real_coms)
        avg_f1 = f1(avg_precision, avg_recall)
        return [avg_recall, avg_precision, avg_f1]
        

In [ ]:
def clusteringwise_sensitivity(reference, predicted):
    num, den = 0., 0.
    for complx in reference:
        den += len(complx)
        num += max(len(complx.intersection(cluster)) for cluster in predicted)
    if den == 0.:
        return 0.
    return num / den

def positive_predictive_value(reference, predicted):
    num, den = 0., 0.
    for cluster in predicted:
        isects = [len(cluster.intersection(compl)) for compl in reference]
        isects.append(0.)
        num += max(isects)
        den += sum(isects)
    if den == 0.:
        return 0.
    return num / den

def sns_ppv_accuracy(reference, predicted):
    sns = clusteringwise_sensitivity(reference, predicted)
    ppv = positive_predictive_value(reference, predicted)
    
    return sns, ppv, (sns * ppv) ** 0.5

<h3> ALL SGD PROTEIN COMPLEX PREDICTION </h3>

In [ ]:
def fracSubset(com1_file, com2_file): #fraction of com1 clusters that are subsets of some com2 clusters
    com1 = read_coms(com1_file)
    com2 = read_coms(com2_file)
    sum1 = 0
    for a in com1:
        for b in com2:
            if set(a) <= set(b):
                sum1 += 1
                break
    ratio = sum1/len(com1)
    return ratio

In [ ]:
def process_all(file, l, theta):
    real_coms_w_names_all = read_real_coms(real_com_file_all, l)
    real_coms_all = [y for [_,y] in real_coms_w_names_all]
    pred_coms = read_coms(file)   
    avg_recall, avg_precision, avg_f1 = prec_rec_f1(real_coms_all, pred_coms)
    real_coms_set = [set(x) for x in real_coms_all]
    pred_coms_set = [set(x) for x in pred_coms]
    #fm_rec = fraction_matched(real_coms_set, pred_coms_set)
    #fm_prec = fraction_matched(pred_coms_set, real_coms_set)
    #fm_f1 = f1(fm_prec, fm_rec) #fm_rec, fm_prec, fm_f1, 
    sns, ppv, acc = sns_ppv_accuracy(real_coms_set, pred_coms_set)
    #print(len(real_coms_all))
    
    #write sns,ppc,acc to csv file for plot generation later
    f = open("plotcsv.txt","a")
    f.write(repr(sns)+','+repr(ppv)+','+repr(acc)+'\n')
    f.close()
    
    return [avg_recall, avg_precision, avg_f1, len(pred_coms)/len(real_coms_all), sns, ppv, acc]

In [ ]:
for th in [0.9, 0.8, 0.7, 0.6]: #, 0.7, 0.6]:
    fpce_com_file = 'fpce-'+species+'-'+str(l)+'-'+str(th)+'-genenames.out'
    cl1_com_file = 'cl1-'+species+'-'+str(l)+'-'+str(th)+'.out'
    print('### RESULTS FOR l='+str(l)+', theta='+str(th))
    res_fpce = process_all(fpce_com_file, l, th)
    res_cl1 = process_all(cl1_com_file, l, th)
    fs = fracSubset(cl1_com_file, fpce_com_file)
    print('FPCE: recall\tprecision\tF1\tfracCom\tSNS\tPPV\tACC\tFS') #recall_fm\tprecision_fm\tF1_fm\t
    print('\t'.join(map(str, res_fpce)), fs, sep = '\t')
    print('CL1: recall\tprecision\tF1\tfracCom\tSNS\tPPV\tACC\tFS')
    print('\t'.join(map(str, res_cl1)))

<h3> 
Find and report some FPCE clusters (for a chosen threshold) which overlaps with some complexes that have no overlap with any CL1 clusters
</h3>

In [ ]:
#select param based on F1 & accuracy
th = 0.8
ov = 1

In [ ]:
def largestOverlappedCommInCom2ForeachCommInCom1(com1, com2):
    maxCommons = []
    i = 1
    for a in com1:
        maxCommon = []
        j = 1
        midx = 0
        for b in com2:
            common = list(set(a) & set(b))
            if len(common) > len(maxCommon):
                maxCommon = common
                midx = j
            j += 1
        if len(maxCommon) > 0:
#             if len(maxCommon) == len(a):
#                 properSubset = []
#             else:
#                 properSubset = a
            maxCommons.append([i, maxCommon, midx, a])
        i += 1

    return maxCommons

In [ ]:
fpce_com_file = 'fpce-'+species+'-'+str(l)+'-'+str(th)+'-genenames.out'
cl1_com_file = 'cl1-'+species+'-'+str(l)+'-'+str(th)+'.out'

real_coms_w_names = read_real_coms(real_com_file_all, l)
real_coms = [y for [_,y] in real_coms_w_names]

fpce_coms = read_coms(fpce_com_file)
cl1_coms = read_coms(cl1_com_file)

#print('RESULTS FOR l='+str(l)+', theta='+str(th))
res_fpce = largestOverlappedCommInCom2ForeachCommInCom1(real_coms, fpce_coms)
res_cl1 = largestOverlappedCommInCom2ForeachCommInCom1(real_coms, cl1_coms)

#[[x,sorted(w),sorted(fpce_coms[z-1]), set(w) <= set(fpce_coms[z-1])] for [x,y,z,w] in res_fpce if len(y) >= 1]
overlappedFPCEclusters = [real_coms_w_names[x-1] for [x,y,z,w] in res_fpce if len(y) >= ov]
overlappedCL1clusters = [real_coms_w_names[x-1] for [x,y,z,w] in res_cl1 if len(y) >= ov]

In [ ]:
len(overlappedCL1clusters)

In [ ]:
len(overlappedFPCEclusters)

In [ ]:
IdxOfRealCommOverlappedFPCEclusters = [x for [x,y,_,_] in res_fpce if len(y)>=ov]
i = 0
for x in IdxOfRealCommOverlappedFPCEclusters:
    print(x, real_coms_w_names[x-1][0], )
    i+=1
i

In [ ]:
len(res_cl1)

In [ ]:
fpceClustNames = set([overlappedFPCEclusters[x][0] for x in range(len(overlappedFPCEclusters))])
cl1ClustNames = set([overlappedCL1clusters[x][0] for x in range(len(overlappedCL1clusters))])
fpce_minus_cl1 = fpceClustNames.difference(cl1ClustNames)
cl1_minus_fpce = cl1ClustNames.difference(fpceClustNames)
print(fpce_minus_cl1, cl1_minus_fpce)

<b>

We see that the following 7-4 = 3 complexes have >=3 common proteins with some FPCE clusters, 
but none of those have any common proteins with any CL1 clusters.
--------------------------------------------
CL#------Real-Community--------maxOverlap-with-FPCE----------maxOverlappedFPCE-cluster
<br>
'C764', ['atpD', 'atpG', 'atpA', 'atpH', 'atpC'],
  ['atpA', 'atpH', 'atpC', 'atpD', 'atpG'],
  ['atpA', 'atpD', 'atpC', 'atpG', 'atpH']
<br>
'C83', ['eno', 'pnp', 'rhlB', 'rne', 'ppk'], ['pnp', 'rne', 'rhlB'], ['rne', 'yfgB', 'pnp', 'rluB', 'srmB', 'rhlB']
<br>
'C909', ['atpC', 'atpH', 'atpA', 'atpG', 'atpD', 'atpE', 'atpF', 'atpB'],
  ['atpA', 'atpE', 'atpC', 'atpD', 'atpG'],
  ['atpA', 'atpD', 'atpC', 'atpG', 'atpE']
</b>

<strong>

Moreover, no elment in cl1_minus_fpce =>

No real complex exists which which has >=1 common proteins with some CL1 cluster, but don't 
have any overlap with any FPCE cluster.

<strong>

In [ ]:
[[real_coms_w_names[x-1], y, fpce_coms[z-1]] for [x,y,z,w] in res_fpce if len(y)>=1]

<h3>
Merge highly overlapping clusters and complete their enrichment analyses
</h3>

In [ ]:
import networkx as nx

In [ ]:
def mergeOverlappingComs(coms, score_threshold=0.2): #default threshold is 0.2 based on Bader et al. 2001
    G = nx.Graph()
    for i in range(len(coms)-1):
        G.add_node(i)
        for j in range(i+1, len(coms)):
            score = matching_score(set(coms[i]), set(coms[j]))
            if score >= score_threshold:
                G.add_edge(i,j)
                break
    G.add_node(len(coms)-1)
    ccs = [cc for cc in nx.connected_components(G)]
    #print(ccs)
    mergedComs = []
    for cc in ccs:
        overlappedComs = [coms[k] for k in cc]
        #print(overlappedComs)
        mergedCom = set().union(*overlappedComs)
        mcm = sorted(mergedCom)
        mergedComs.append(list(mcm))
    return mergedComs

In [ ]:
fpce_mcoms = mergeOverlappingComs(fpce_coms)
with open('fpce-ov-graph-ccs.txt','w') as f:
    for i in fpce_mcoms:
        f.write(' '.join(i))
        f.write('\n')

In [ ]:
! ln=1;cat fpce-ov-graph-ccs.txt | while read line; do echo $line | tr ' ' '\n' > tmp.study; python goatools-main/scripts/find_enrichment.py tmp.study e-coli-data/ecoli-all.pop e-coli-data/ecocyc-gene2golist.txt --outfile=goea-fpce-ov-graph-comm-"$ln".tsv --pval_field=fdr_bh --ns=CC --ev_exc=IEA --pval=0.05; ((ln+=1)); done

In [ ]:
cl1_mcoms = mergeOverlappingComs(cl1_coms)
with open('cl1-ov-graph-ccs.txt','w') as f:
    for i in cl1_mcoms:
        f.write(' '.join(i))
        f.write('\n')

In [ ]:
! ln=1;cat cl1-ov-graph-ccs.txt | while read line; do echo $line | tr ' ' '\n' > tmp.study; python goatools-main/scripts/find_enrichment.py tmp.study e-coli-data/ecoli-all.pop e-coli-data/ecocyc-gene2golist.txt --outfile=goea-cl1-ov-graph-comm-"$ln".tsv --pval_field=fdr_bh --ns=CC --ev_exc=IEA --pval=0.05; ((ln+=1)); done

In [ ]:
print(len(cl1_mcoms), len(fpce_mcoms))

In [ ]:
! ls -1 goea-cl1-ov-graph-comm-* |wc -l

In [ ]:
! ls -1 goea-fpce-ov-graph-comm-* |wc -l

In [ ]:
print("%enriched among CL1 merged clusters: ", 100*8/13)
print("%enriched among FPCE merged clusters: ", 100*15/18)

In [ ]:
! grep -hv \# goea-cl1-ov-graph-comm-*.tsv | cut -f1 | sort | uniq > cl1-enriched-uniq-go-terms.txt
! grep -hv \# goea-fpce-ov-graph-comm-*.tsv | cut -f1 | sort | uniq > fpce-enriched-uniq-go-terms.txt

In [ ]:
def enrichedTerms(file):
    with open(file) as file:
        eterms = [line.rstrip() for line in file]
    return eterms

In [ ]:
cl1_en_terms = enrichedTerms('cl1-enriched-uniq-go-terms.txt')
fpce_en_terms = enrichedTerms('fpce-enriched-uniq-go-terms.txt')
len(cl1_en_terms)

In [ ]:
fpce_minus_cl1_et = set(fpce_en_terms).difference(cl1_en_terms)
len(fpce_minus_cl1_et)

In [ ]:
cl1_minus_fpce_et = set(cl1_en_terms).difference(fpce_en_terms)
cl1_minus_fpce_et #### FIND which are these two, are they ancestors of some GO terms enriched in FPCE??? In that case, they add no new info